In [8]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer,TextClassificationPipeline

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [2]:
import shap

In [3]:
import transformers

In [4]:
from datasets import load_dataset
raw_datasets=load_dataset('csv',data_files={'train': 'train.csv',
                                              'test':'test.csv'})

Using custom data configuration default-0465f68cbeafdcaf
Reusing dataset csv (C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 2/2 [00:00<00:00, 34.60it/s]


In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200000
    })
})

In [5]:
'''model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)'''

'model_name = "siebert/sentiment-roberta-large-english"\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name)\ntrainer = Trainer(model=model)'

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [9]:
tokenizer_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)
model = transformers.DistilBertForSequenceClassification.from_pretrained(model_name).to('cuda')
trainer = Trainer(model=model)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-14b885d58d73eded.arrow
Loading cached processed dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-c5a5541ea1f4db19.arrow


In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=10).select(range(2000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=10).select(range(200)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-69653c1aeeb3edb3.arrow
Loading cached shuffled indices for dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-0fe216909a5212df.arrow


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate = 2e-5,
    #batch_size = 32,
    warmup_steps=0 ,
    #max_seq_length = 128,
    num_train_epochs = 5.0 ,
    weight_decay=0.01 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [34]:
torch.cuda.is_available()

True

In [35]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
 40%|████      | 500/1250 [00:45<01:09, 10.80it/s]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 0.0528, 'learning_rate': 1.2e-05, 'epoch': 2.0}


Model weights saved in ./results\checkpoint-500\pytorch_model.bin
 80%|████████  | 1000/1250 [01:33<00:23, 10.47it/s]Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json


{'loss': 0.0214, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.0}


Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
100%|██████████| 1250/1250 [01:57<00:00, 10.70it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1250/1250 [01:57<00:00, 10.63it/s]

{'train_runtime': 117.6415, 'train_samples_per_second': 85.004, 'train_steps_per_second': 10.626, 'train_loss': 0.03214371929168701, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.03214371929168701, metrics={'train_runtime': 117.6415, 'train_samples_per_second': 85.004, 'train_steps_per_second': 10.626, 'train_loss': 0.03214371929168701, 'epoch': 5.0})

In [36]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
100%|██████████| 25/25 [00:00<00:00, 34.93it/s]


{'eval_loss': 1.5619434118270874,
 'eval_accuracy': 0.855,
 'eval_runtime': 1.2852,
 'eval_samples_per_second': 155.62,
 'eval_steps_per_second': 19.452}

In [39]:
trainer.save_model('C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english')

Saving model checkpoint to C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english
Configuration saved in C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english\config.json
Model weights saved in C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english\pytorch_model.bin


Saved Model

In [5]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig, AutoModelForTokenClassification

# retreive the saved model 
model = DistilBertForSequenceClassification.from_pretrained('C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english', 
                                                        local_files_only=True)

model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [8]:
tokenizer_name = "distilbert-base-uncased"
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)

In [12]:
from transformers import TextClassificationPipeline
pipe=TextClassificationPipeline(tokenizer=tokenizer,model=model,device=0,return_all_scores=True)

In [21]:
'''from transformers import pipeline
pipe=pipeline("sentiment-analysis",tokenizer=tokenizer,model=model,device=0)'''

'from transformers import pipeline\npipe=pipeline("sentiment-analysis",tokenizer=tokenizer,model=model,device=0)'

In [13]:
text="study interventions are Immunoglobulins . solid neoplasm diagnosis and subjects taking strong cytochrome pfour hundred and fifty family three subfamily polypeptide four cypthreeafour and cytochrome pfour hundred and fifty family two subfamily polypeptide nineteen cyptwocnineteen inhibitors and or inducers should be considered with caution alternative treatments that are less likely to affect mln0onetwo8 metabolism if available should be considered if subject requires treatment with one or more of the strong cypthreeafour and cyptwocnineteen inhibitors and or inducers the principal investigator should be consulted,"

In [14]:
pipe(text)

[[{'label': 'NEGATIVE', 'score': 0.09999211877584457},
  {'label': 'POSITIVE', 'score': 0.9000079035758972}]]

In [15]:
cor_x = np.load("test2.npy",allow_pickle=True)
cor_reviews = [review[0] for review in cor_x]
cor_labels = [review[1] for review in cor_x]

In [17]:
cor_x[0]

array(['study interventions are Immunosuppressive Agents . contiguous stage ii adult lymphoblastic lymphoma diagnosis and treatment with rituximab for any reason in the twelve months preceding hct',
       1], dtype=object)

In [18]:
def score_and_visualize(text):
  prediction = pipe([text])
  print(prediction[0])

  explainer = shap.Explainer(pipe)
  shap_values = explainer([text])

  shap.plots.text(shap_values)

In [26]:
'''tokenizer_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)
model = transformers.DistilBertForSequenceClassification.from_pretrained(model_name).to('cuda')

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)'''

'tokenizer_name = "distilbert-base-uncased"\nmodel_name = "distilbert-base-uncased-finetuned-sst-2-english"\n\ntokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)\nmodel = transformers.DistilBertForSequenceClassification.from_pretrained(model_name).to(\'cuda\')\n\npipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)'

In [19]:
score_and_visualize('After many years of training, I finally succeeded in achieving '
                    'my black belt in machine learning. The road was very rough but '
                    'I could not be more happy with the result!')

[{'label': 'NEGATIVE', 'score': 0.011526159942150116}, {'label': 'POSITIVE', 'score': 0.9884738326072693}]


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [28]:
'''explainer = shap.Explainer(pipe)
selected_labels =[]
top_k = 1
for review, label in zip(cor_reviews, cor_labels):
    # to-do: truncate review if len(review)>80
    prediction = pipe([review])
    if prediction[0]['label'] == 'NEGATIVE':
            neg_score = prediction[0]["score"]
            pos_score=1-prediction[0]["score"]
    else:
            pos_score = prediction[0]["score"] 
            neg_score = 1-prediction[0]["score"]       
        pred_label = 0 if prediction[0]['label'] == 'NEGATIVE'  else 1
    if pred_label == label:
        selected_labels.append(label)
        shap_values = explainer([text])
        values = shap_values.values # 2-dim ndarray
        data_tokens = shap_values.data
        if prediction[0]['label'] == 'NEGATIVE':
            neg_score = prediction[0]["score"]
            pos_score=1-prediction[0]["score"]
        else:
            pos_score = prediction[0]["score"] 
            neg_score = 1-prediction[0]["score"]       
        pred_label = 0 if prediction[0]['label'] == 'NEGATIVE'  else 1

        neg_shap_values, pos_shap_values = zip(*shap_values)
        MAXLEN = 80

        #truncate the values
        truncated_neg_value =neg_shap_values[:80]
        truncated_pos_value =pos_shap_values[:80]

        top_k_ids = (-arr).argsort()[:top_k]
        trunctated_tokens = data_tokens[:80]
        masks = np.zeros(truncated_pos_value.shape, bool)
        for index in len(trunctated_tokens):
            if trunctated_tokens[index] in top_k_ids:
                trunctated_tokens[index] = " "
        



res =""
for word_list in shap_values.data:
    for word in word_list:
        res += word
print(res) '''

'explainer = shap.Explainer(pipe)\nselected_labels =[]\ntop_k = 1\nfor review, label in zip(cor_reviews, cor_labels):\n    # to-do: truncate review if len(review)>80\n    prediction = pipe([review])\n    if prediction[0][\'label\'] == \'NEGATIVE\':\n            neg_score = prediction[0]["score"]\n            pos_score=1-prediction[0]["score"]\n    else:\n            pos_score = prediction[0]["score"] \n            neg_score = 1-prediction[0]["score"]       \n        pred_label = 0 if prediction[0][\'label\'] == \'NEGATIVE\'  else 1\n    if pred_label == label:\n        selected_labels.append(label)\n        shap_values = explainer([text])\n        values = shap_values.values # 2-dim ndarray\n        data_tokens = shap_values.data\n        if prediction[0][\'label\'] == \'NEGATIVE\':\n            neg_score = prediction[0]["score"]\n            pos_score=1-prediction[0]["score"]\n        else:\n            pos_score = prediction[0]["score"] \n            neg_score = 1-prediction[0]["scor

In [29]:
def mask_top_k(k, pred_label_no_mask, values, returned_tokens):
    """
    masked the k tokens that have the max shap values
    :param k: specify the largest k value
    :param values: shap values
    :param returned_tokens: a list of tokens
    :return: review, which is a str constructed from a list words
    """
    shap_values_neg, shap_values_pos = zip(*values)
    values = shap_values_neg if pred_label_no_mask==0 else shap_values_pos
    # print(values)
    values = np.array(values)
    # ids_top_k = np.argpartition(values, -k)[-k:]
    ids_top_k = (-values).argsort()[:k]
    for idx in ids_top_k:
        # print(idx)
        returned_tokens[idx] = "[UNK] "
    masked_review = "".join(returned_tokens)
    # print(masked_review)
    return masked_review

In [30]:
def predict_label(pipe, masked_review):
    """
    predict the label for the masked_review
    :param pipe: pipeline
    :param masked_review: string
    :return: 0 or 1, indicating the label
    """
    prediction = pipe([masked_review])
    if prediction[0]['label'] == 'NEGATIVE':
            neg_score = prediction[0]["score"]
            pos_score=1-prediction[0]["score"]
    else:
            pos_score = prediction[0]["score"] 
            neg_score = 1-prediction[0]["score"]       
    pred_label = 0 if prediction[0]['label'] == 'NEGATIVE'  else 1
    return pred_label

In [31]:
'''def predict_label(pipe, masked_review):
    """
    predict the label for the masked_review
    :param pipe: pipeline
    :param masked_review: string
    :return: 0 or 1, indicating the label
    """
    prediction = pipe([masked_review])
    neg_prediction_score = prediction[0][0]["score"]
    pos_prediction_score = prediction[0][1]["score"]
    pred_label = 0 if neg_prediction_score > pos_prediction_score else 1
    return pred_label'''

'def predict_label(pipe, masked_review):\n    """\n    predict the label for the masked_review\n    :param pipe: pipeline\n    :param masked_review: string\n    :return: 0 or 1, indicating the label\n    """\n    prediction = pipe([masked_review])\n    neg_prediction_score = prediction[0][0]["score"]\n    pos_prediction_score = prediction[0][1]["score"]\n    pred_label = 0 if neg_prediction_score > pos_prediction_score else 1\n    return pred_label'

In [33]:
def model_prediction_gpu(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', 
                                        max_length=90, truncation=True) for v in x]).cuda()
    attention_mask = (tv!=0).type(torch.int64).cuda()
    outputs = model(tv, attention_mask=attention_mask)[0]
    scores = torch.nn.Softmax(dim=-1)(outputs)
    val = torch.logit(scores).detach().cpu().numpy()

    return val

In [34]:
'''gpu_explainer = shap.Explainer(model_prediction_gpu, tokenizer)'''

'gpu_explainer = shap.Explainer(model_prediction_gpu, tokenizer)'

In [35]:
'''gpu_explainer(review)'''

'gpu_explainer(review)'

In [36]:
'''import transformers
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          TextClassificationPipeline)'''

In [37]:
'''tokenizer_name2 = "distilbert-base-uncased"
model_name2 = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer2 = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name2)
model2 = transformers.DistilBertForSequenceClassification.from_pretrained(model_name2).to('cuda')

pipe2 = TextClassificationPipeline(model=model2, tokenizer=tokenizer2, return_all_scores=True,device=0)'''

'tokenizer_name2 = "distilbert-base-uncased"\nmodel_name2 = "distilbert-base-uncased-finetuned-sst-2-english"\n\ntokenizer2 = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name2)\nmodel2 = transformers.DistilBertForSequenceClassification.from_pretrained(model_name2).to(\'cuda\')\n\npipe2 = TextClassificationPipeline(model=model2, tokenizer=tokenizer2, return_all_scores=True,device=0)'

In [38]:
'''tv=torch.tensor([tokenizer.encode(v, padding='max_length', 
                                        max_length=90, truncation=True) for v in review]).cuda()'''

"tv=torch.tensor([tokenizer.encode(v, padding='max_length', \n                                        max_length=90, truncation=True) for v in review]).cuda()"

In [39]:
'''attention_mask = (tv!=0).type(torch.int64).cuda()'''

'attention_mask = (tv!=0).type(torch.int64).cuda()'

In [40]:
shap_values_list = []
token_data_list = []
top_k = [1, 5, 9, 13]
all_labels =[]
# use GPU
gpu_explainer = shap.Explainer(model_prediction_gpu, tokenizer)
i = 0
for review, label in zip(cor_reviews, cor_labels):
    print(f"process {i}-th review")
    i += 1
    label4review =[]
    label4review.append(label)
    # to-do: truncate review if len(review)>80
    tokens = tokenizer.tokenize(review)
    if len(tokens) > 80:
        tokens_truncated = tokens[:80]
        review = " ".join(token for token in tokens_truncated)
    pred_label_no_mask = predict_label(pipe, review) # predicted label for review without mask
    label4review.append(pred_label_no_mask)
    shap_values = gpu_explainer(review)
    values = shap_values.values[0] # 2-dim ndarray
    returned_tokens = shap_values.data[0]
    for k in top_k:
        masked_review = mask_top_k(k, pred_label_no_mask, values, returned_tokens) # mask review by the shap values
        predicted_label= predict_label(pipe, masked_review)
        label4review.append(predicted_label)
    # label4review = [True_label, pred_label_without_mask, masked_label_1, masked_label_2, masked_review_3, masked_review_4]
    all_labels.append(label4review)
    with open("labels.csv", mode="a") as f:
        f.write("%s\n" % ",".join(str(label) for label in label4review))


process 0-th review


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


process 1-th review
process 2-th review
process 3-th review
process 4-th review
process 5-th review
process 6-th review
process 7-th review
process 8-th review
process 9-th review
process 10-th review
process 11-th review
process 12-th review
process 13-th review
process 14-th review
process 15-th review
process 16-th review
process 17-th review
process 18-th review
process 19-th review
process 20-th review
process 21-th review
process 22-th review
process 23-th review
process 24-th review
process 25-th review
process 26-th review
process 27-th review
process 28-th review
process 29-th review
process 30-th review
process 31-th review
process 32-th review
process 33-th review
process 34-th review
process 35-th review
process 36-th review
process 37-th review
process 38-th review
process 39-th review
process 40-th review
process 41-th review
process 42-th review
process 43-th review
process 44-th review
process 45-th review
process 46-th review
process 47-th review
process 48-th review
p

Partition explainer: 158it [08:44,  4.23s/it]                         


process 1997-th review
process 1998-th review
process 1999-th review


In [41]:
np.save("all_labels.npy", np.array(all_labels))

In [42]:
true_label, predicted_label_without_mask, predicted_label_mask1, predicted_label_mask5, predicted_label_mask9, predicted_label_mask13 = zip(*all_labels)

In [43]:
true_label =[int(ele) for ele in true_label]

In [44]:
import pandas as pd
df = pd.DataFrame(np.array(all_labels))

In [45]:
filtered_df = df.loc[df[0]==df[1]]
len(filtered_df)

1559

In [46]:
from sklearn.metrics import accuracy_score

In [47]:
print(accuracy_score(filtered_df[1], filtered_df[2]), accuracy_score(filtered_df[1], filtered_df[3]), accuracy_score(filtered_df[1], filtered_df[4]), accuracy_score(filtered_df[1], filtered_df[5]))

0.4733803720333547 0.4733803720333547 0.4733803720333547 0.4733803720333547


In [48]:
accuracy_score(df[0],df[1])

0.7795

In [49]:
'''import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd'''

'import numpy as np\nfrom sklearn.model_selection import train_test_split\nimport csv\nimport pandas as pd'

In [50]:
#alltest=pd.read_csv('test.csv')

In [2]:
from transformers import DistilBertForSequenceClassification
model2 = DistilBertForSequenceClassification.from_pretrained('C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english',
                                                        local_files_only=True)
model2.to('cuda')
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

ModuleNotFoundError: No module named 'transformers'

In [51]:
#test1,test2=train_test_split(alltest, test_size=0.01, random_state=101010)

In [52]:
#np.save('test2.npy',test2)